In [5]:
from getData import df
import pandas as pd

In [7]:
df.head()

,Datetime,Author,Conversation,Sent Messages,Sent Words,WPM,Total Messages,Total Words
1129,2018-09-28 03:37:00,Miles Keating,Neighbor,1,8,8.0,1,8
1130,2018-09-28 14:28:00,Neighbor,Neighbor,1,5,5.0,2,13
1131,2018-09-28 14:31:00,Miles Keating,Neighbor,2,10,5.0,3,15
1132,2018-09-28 14:49:00,Neighbor,Neighbor,2,6,3.0,4,16
1133,2018-11-01 12:00:00,Miles Keating,Neighbor,3,18,6.0,5,24


# tab4 pm by conversation

# tab3 groupchats by conversation

In [9]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, Legend, LegendItem
from bokeh.models.widgets import RadioButtonGroup, Select, CheckboxGroup
from bokeh.layouts import column, row
from bokeh.palettes import turbo, Category20_20, Category10_10

output_notebook()

#Organize data

def tab1(doc):
    global df
    df = df.drop(df[df.Conversation.isin(list(df.Author.unique()))].index)
    df = df[['Datetime', 'Author','Conversation', 'Total Messages', 'Total Words']]
    
    me = 'Miles Keating'
    reset_data={x:[] for x in df.columns.unique()}
    conversations = list(df.Conversation.unique())

    if len(conversations) > 20:
        colors = turbo(len(subset.Conversation.unique()))
    elif len(conversations) > 10:
        colors = Category20_20
    else:
        colors = Category10_10
    
    #Create Figures
    height = 300
    width = 1000
    
    top = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    legend = Legend(items=[])
    top.add_layout(legend, 'right')
    bot = figure(x_axis_type='datetime', frame_height=height, frame_width=width)

    



    def update_conversation(attr, old, new): #Updates conversation from checkbox group
        subset = df
        if len(old) > len(new): #signifies unchecking a box
            change = checkbox.labels[list(set(old) - set(new))[0]]
            new_data = reset_data
        else: #signifies checking a box
            index = list(set(new) - set(old))[0]
            change = checkbox.labels[list(set(new) - set(old))[0]]
            new_data = subset[subset.Conversation == change]
            renders[change].glyph.line_color=colors[index]
        items=[]
        for x in checkbox.active:
            x= checkbox.labels[x]
            items.append((x,[renders[x]]))
        top.legend.items=items

        src[change].data.update(new_data)
        

    def update_column(attr, old, new):
        newval = column_radio.labels[new]
        for r in renders:
            renders[r].glyph.y = newval
            
        #Create Widgets
    column_selection = ['Total Messages', 'Total Words']
    column_radio = RadioButtonGroup(labels= column_selection,
                                   active=0)

    

    subset = df

    # Create Checkbox Widget

    checkbox = CheckboxGroup(labels=conversations, active=[])
    
    legend_items =[]
    #Populate the src and tie each one to a render
    src = {} #Dictionary of CDS' where each author maps to a CDS
    renders = {} # Dictionary of renderers where each author maps to a renderer
    for conversation in conversations:
        src[conversation] = ColumnDataSource(reset_data)
        renders[conversation] = top.line(source=src[conversation],
                           x='Datetime', y = column_radio.labels[column_radio.active])
        legend_items.append([(conversation,[renders[conversation]])])
        
    
    top.legend.items = []

    checkbox.on_change('active', update_conversation)
    column_radio.on_change('active', update_column)
    
    layout = column(column_radio, row(checkbox, top))
    doc.add_root(layout)

            
show(tab1)

print('done')

Loading BokehJS ...

done


# tab2 pmchats by author

In [4]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, Legend, LegendItem
from bokeh.models.widgets import RadioButtonGroup, Select, CheckboxGroup
from bokeh.layouts import column, row

output_notebook()

#Organize data

def tab1(doc):
    
    me = 'Miles Keating'
    global df
    reset_data = {x:[] for x in df.columns.unique()}
    
    colors = ['navy','firebrick']
    
    #Create Figures
    height = 300
    width = 1000
    
    top = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    legend = Legend(items=[])
    top.add_layout(legend, 'right')
    bot = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    
    df = df.drop(df[~df.Conversation.isin(list(df.Author.unique()))].index)
    


    def update_author(attr, old, new):
        subset = df[df.Conversation == conversation_select.value]
        
        if len(old) > len(new): #signifies unchecking a box
            change = checkbox.labels[list(set(old) - set(new))[0]]
            new_data = reset_data
        else: #signifies checking a box
            index = list(set(new) - set(old))[0]
            change = checkbox.labels[list(set(new) - set(old))[0]]
            new_data = subset[subset.Author == change]
            renders[change].glyph.line_color=colors[index]

        top.legend.items=reset_legend(checkbox.labels[checkbox.active])

        src[change].data.update(new_data)
        
    def update_conversation(attr, old, new):
        subset = df[df.Conversation == new]
        
        for author in all_authors:
            src[author].data.update(reset_data)
            if author in subset.Author.unique():
                src[author].data.update(subset[subset.Author == author])
            
        checkbox.active = [0,1]
        checkbox.labels = list(subset.Author.unique())
        top.legend.items=reset_legend([checkbox.labels[x] for x in checkbox.active])

        
    def reset_legend(authors):
        items = []
        for author in authors:
            items.append((author,[renders[author]]))
        return items

    def update_column(attr, old, new):
        newval = column_radio.labels[new]
        for r in renders:
            renders[r].glyph.y = newval

    all_authors = df.Author.unique()
    conversations = list(df.Conversation.unique())
    
    #Create Widgets
    column_selection = ['Sent Messages', 'Sent Words','WPM']
    column_radio = RadioButtonGroup(labels= column_selection,
                                   active=1)
    conversation_select = Select(title='Conversation',
                                value =conversations[0],
                                options=conversations)
    
    subset = df[df.Conversation == conversation_select.value]
    authors = list(subset.Author.unique())

    # Create Checkbox Widget

    checkbox = CheckboxGroup(labels=authors, active=[0,1])
    
    #Populate the src and tie each one to a render
    src = {} #Dictionary of CDS' where each author maps to a CDS
    renders = {} # Dictionary of renderers where each author maps to a renderer
    items = []
    for author in all_authors:
        src[author] = ColumnDataSource(reset_data)
        renders[author] = top.line(source=src[author],
                           x='Datetime', y = column_radio.labels[column_radio.active], line_color=colors[1])
        if author in authors:
            src[author].data.update(subset[subset.Author == author])
            if author == me:
                renders[me].glyph.line_color=colors[0]
            items.append((author,[renders[author]]))
    top.legend.items=items
    
    checkbox.on_change('active', update_author)
    conversation_select.on_change('value', update_conversation)
    column_radio.on_change('active', update_column)
    
    layout = column(column_radio,
                    row(column(conversation_select, checkbox), top))
    doc.add_root(layout)


    
show(tab1)

print('done')

Loading BokehJS ...

done


# tab1

Loading BokehJS ...

done


ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=1002&bokeh-absolute-url=http://localhost:35861&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:35861', method='GET', uri='/autoload.js?bokeh-autoload-element=1002&bokeh-absolute-url=http://localhost:35861&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/home/miles/pydir/envy/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: ('Datetime', 'Conversation', 'Total Messa